In [49]:
import torch 
import torch.nn as nn
from torch.utils.data import  DataLoader
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import os
from typing import Tuple, Dict, List
from PIL import Image
from pathlib import Path
from tqdm.auto import tqdm
# https://medium.com/@karuneshu21/resnet-paper-walkthrough-b7f3bdba55f0
# https://arxiv.org/pdf/1512.03385.pdf
# https://medium.com/@karuneshu21/how-to-resnet-in-pytorch-9acb01f36cf5
torch.autograd.set_detect_anomaly(True)

In [50]:
class ImageFolderCustom(Dataset):
    
    # 2. Initialize with a targ_dir and transform (optional) parameter
    def __init__(self, targ_dir: str, transform=None) -> None:
        
        # 3. Create class attributes
        # Get all image paths
        self.paths = list(Path(targ_dir).glob("*/*.jpg")) # note: you'd have to update this if you've got .png's or .jpeg's
        # Setup transforms
        self.transform = transform
        # Create classes and class_to_idx attributes
        # print(self.paths)
        self.classes, self.class_to_idx = self.find_classes(targ_dir)

    # 4. Make function to load images
    def load_image(self, index: int) -> Image.Image:
        "Opens an image via a path and returns it."
        image_path = self.paths[index]
        return Image.open(image_path) 
    
    # 5. Overwrite the __len__() method (optional but recommended for subclasses of torch.utils.data.Dataset)
    def __len__(self) -> int:
        "Returns the total number of samples."
        return len(self.paths)
    
    # 6. Overwrite the __getitem__() method (required for subclasses of torch.utils.data.Dataset)
    def __getitem__(self, index: int) -> Tuple[torch.Tensor, int]:
        "Returns one sample of data, data and label (X, y)."
        img = self.load_image(index)
        class_name  = self.paths[index].parent.name # expects path in data_folder/class_name/image.jpeg
        class_idx = self.class_to_idx[class_name]

        # Transform if necessary
        if self.transform:
            return self.transform(img), class_idx # return data, label (X, y)
        else:
            return img, class_idx # return data, label (X, y)
        
    def find_classes(self,directory:str)->Tuple[list[str],dict[str,int]]:
        classes=sorted(entry.name for entry in os.scandir(directory) if entry.is_dir())

        if not classes:
            raise FileNotFoundError(f"Couldn't find any classes in {directory}.")
        
        class_to_idx={cls_name: i for i, cls_name in enumerate(classes)}
        
        print(classes,class_to_idx)
        return classes,class_to_idx


In [51]:
class CustomDataTest():
    def __init__(self) -> None:

        self.train_dir = "C:\\Users\\Jakub Machura\\source\\repos\\UnderstandingDeepLearning\\data\\pizza_steak_sushi\\train"
        self.test_dir = "C:\\Users\\Jakub Machura\\source\\repos\\UnderstandingDeepLearning\\data\\pizza_steak_sushi\\test"
        """
            temp placement for of transform
        """
        self.train_transform=transforms.Compose([
            transforms.Resize(size=(224,224)),
            # transforms.TrivialAugmentWide(num_magnitude_bins=31),
            transforms.ToTensor()
        ])
    
        self.test_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
        ])
    
        self.train_data =ImageFolderCustom(self.train_dir,transform=self.train_transform)
        self.test_data=ImageFolderCustom(self.test_dir,transform=self.test_transforms)
    
        """temp place for testing data"""    
        # Check for equality amongst our custom Dataset and ImageFolder Dataset
        # print((len(self.train_data) == len(data.train_data)) & (len(self.test_data) == len(data.test_data)))
        print(self.train_data.classes)
        # print(self.train_data.class_to_idx == data.train_data.class_to_idx)

        self.IntoDataLoaders()

    def IntoDataLoaders(self):
        BATCH_SIZE=32
        # NUM_WORKERS = os.cpu_count()
        NUM_WORKERS = 1

        # print(f"number of workers avalible {NUM_WORKERS}")
        self.train_dataloader= DataLoader(dataset=self.train_data, # use custom created train Dataset
                                     batch_size=BATCH_SIZE, # how many samples per batch?
                                    #  num_workers=NUM_WORKERS, # how many subprocesses to use for data loading? (higher = more)
                                     shuffle=True) # shuffle the data?

        self.test_dataloader = DataLoader(dataset=self.test_data, # use custom created test Dataset
                                    batch_size=BATCH_SIZE, 
                                    # num_workers=NUM_WORKERS, 
                                    shuffle=False) # don't usually need to shuffle testing data

        img,label=next(iter(self.test_dataloader))
        print(f"shape of custome dataloader img {img.shape} labels {label.shape}")

In [52]:
class ResNet(nn.Module):
    def __init__(self,in_channels,num_classes) -> None:
        super().__init__()

        self.conv1=nn.Conv2d(in_channels,64,kernel_size=7,stride=2,padding=3)
        self.bn1=nn.BatchNorm2d(64)
        self.relu=nn.ReLU()
        self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)

        self.avgpool=nn.AdaptiveAvgPool2d((1,1))
        self.fc=nn.Linear(512,num_classes)

        self.in_channels=64
        self.out_channels=64

        # self.conv2=[]
        # for i in range(layers[0]):
        #     self.conv2.append(nn.Conv2d(self.in_channels,self.out_channels,kernel_size=3,padding=2,stride=1))
        # self.bn2=nn.BatchNorm2d(self.out_channels)

        self.conv2_1=nn.Conv2d(self.in_channels,self.out_channels,kernel_size=3,padding=1,stride=1)
        self.bn2_1=nn.BatchNorm2d(self.out_channels)

        self.out_channels=self.out_channels*2
        self.conv3_1=nn.Conv2d(self.in_channels,self.out_channels,kernel_size=3,padding=1,stride=2)
        self.bn3_1=nn.BatchNorm2d(self.out_channels)

        self.in_channels=self.out_channels
        self.out_channels=self.out_channels*2
        self.conv4_1=nn.Conv2d(self.in_channels,self.out_channels,kernel_size=3,padding=1,stride=2)
        self.bn4_1=nn.BatchNorm2d(self.out_channels)


        self.in_channels=self.out_channels
        self.out_channels=self.out_channels*2
        self.conv5_1=nn.Conv2d(self.in_channels,self.out_channels,kernel_size=3,padding=1,stride=2)
        self.bn5_1=nn.BatchNorm2d(self.out_channels)

    def forward(self,x):
       

        x=self.conv1(x)
        print(f"after conv1 channels: 64 kernel_size: 7 stride: 2 padding: 3 {x.shape}")
        x=self.bn1(x)
        x=self.relu(x)
        x=self.maxpool(x)
        print(f"after maxpool kernel_size: 3 stride: 2 padding: 1 {x.shape}")
        identity=x
        x=self.conv2_1(x)
        x=self.bn2_1(x)
        x=self.relu(x)
        x+=identity
        print(f"identity shape {identity.shape}")
        print(f"after conv2 channels: 64 kernel_size: 3 stride: 1 padding: 1 {x.shape}")


        identity=x
        x=self.conv3_1(x)
        x=self.bn3_1(x)
        x=self.relu(x)
        identity_downsample=nn.Sequential(
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=1,stride=2,padding=0),
            nn.BatchNorm2d(num_features=128)
        )
        x+=identity_downsample(identity)
        print(f"identity shape {identity.shape}")
        print(f"after conv3 channels: 128 kernel_size: 3 stride: 2 padding: 1 {x.shape}")

        
        
        identity=x
        x=self.conv4_1(x)
        x=self.bn4_1(x)
        x=self.relu(x)
        identity_downsample=nn.Sequential(
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=1,stride=2,padding=0),
            nn.BatchNorm2d(num_features=256)
        )
        x+=identity_downsample(identity)
        print(f"identity shape {identity.shape}")
        print(f"after conv4 channels: 256 kernel_size: 3 stride: 2 padding: 1 {x.shape}")
        

        identity=x
        x=self.conv5_1(x)
        x=self.bn5_1(x)
        x=self.relu(x)
        identity_downsample=nn.Sequential(
            nn.Conv2d(in_channels=256,out_channels=512,kernel_size=1,stride=2,padding=0),
            nn.BatchNorm2d(num_features=512)
        )
        x+=identity_downsample(identity)
        print(f"identity shape {identity.shape}")
        print(f"after conv4 channels: 256 kernel_size: 3 stride: 2 padding: 1 {x.shape}")
        
        x=self.avgpool(x)
        print(f"after avgpool {x.shape}")
        x=x.reshape(x.shape[0],-1)
        print(f"after reshape {x.shape}")

        x=self.fc(x)
        
        return x

    # def __make_layers(self,num_residual_block,out_channels,stride):
        
    #     layers=[]
    #     for i in range(num_residual_block):
    #         layers.append(nn.Conv2d(self.in_channels,out_channels,kernel_size=3,padding=2,stride=stride))
        
    #     return(nn.Sequential(*layers))

In [76]:
def train_step(model,optimizer,loss_fn,data:CustomDataTest):
    
    model.train()
    train_loss=0

    for batch, (X,y) in enumerate(data.train_dataloader):
        y_logits=model(X)

        loss=loss_fn(y_logits,y)
        train_loss+=loss.item()


        optimizer.zero_grad()

        loss.backward()

        optimizer.step()
        
    train_loss=train_loss/len(data.train_dataloader)
    return train_loss

def test_step(model,loss_fn,data:CustomDataTest):
    model.eval()
    test_loss=0

    with torch.inference_mode():
        for batch, (X,y) in enumerate(data.test_dataloader):
            y_logits=model(X)
            loss=loss_fn(y_logits,y)
            test_loss+=loss.item()

    test_loss=test_loss/len(data.test_dataloader)
    return test_loss

def Totrain(model,data:CustomDataTest,optimizer,loss_fn,epochs:int):
    results={"train_loss":[],
             "test_loss":[]}
    
    for epoch in tqdm(range(epochs)):
        train_loss=train_step(model,optimizer,loss_fn,data)
        test_loss=test_step(model,loss_fn,data)

                
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"test_loss: {test_loss:.4f} | "
        )

        # 5. Update results dictionary
        results["train_loss"].append(train_loss)
        results["test_loss"].append(test_loss)
    
    return results

In [55]:
data=CustomDataTest()

net=ResNet(3,len(data.train_data.classes))
net_optimizer=torch.optim.SGD(net.parameters(),lr=0.1)
net_loss_fn=nn.CrossEntropyLoss()
res=Totrain(net,data,net_optimizer,net_loss_fn,2)


# x=torch.randn(32,3,224,224)
# print(f"out shape {y.shape}")



['pizza', 'steak', 'sushi'] {'pizza': 0, 'steak': 1, 'sushi': 2}
['pizza', 'steak', 'sushi'] {'pizza': 0, 'steak': 1, 'sushi': 2}
['pizza', 'steak', 'sushi']
shape of custome dataloader img torch.Size([32, 3, 224, 224]) labels torch.Size([32])


  0%|          | 0/2 [00:00<?, ?it/s]

after conv1 channels: 64 kernel_size: 7 stride: 2 padding: 3 torch.Size([32, 64, 112, 112])
after maxpool kernel_size: 3 stride: 2 padding: 1 torch.Size([32, 64, 56, 56])
identity shape torch.Size([32, 64, 56, 56])
after conv2 channels: 64 kernel_size: 3 stride: 1 padding: 1 torch.Size([32, 64, 56, 56])
identity shape torch.Size([32, 64, 56, 56])
after conv3 channels: 128 kernel_size: 3 stride: 2 padding: 1 torch.Size([32, 128, 28, 28])
identity shape torch.Size([32, 128, 28, 28])
after conv4 channels: 256 kernel_size: 3 stride: 2 padding: 1 torch.Size([32, 256, 14, 14])


  0%|          | 0/2 [00:00<?, ?it/s]

identity shape torch.Size([32, 256, 14, 14])
after conv4 channels: 256 kernel_size: 3 stride: 2 padding: 1 torch.Size([32, 512, 7, 7])
after avgpool torch.Size([32, 512, 1, 1])
after reshape torch.Size([32, 512])


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [32, 512, 7, 7]], which is output 0 of ReluBackward0, is at version 1; expected version 0 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [32, 512, 7, 7]], which is output 0 of ReluBackward0, is at version 1; expected version 0 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!


In [59]:
class SimpleResNetblock(nn.Module):
    def __init__(self,in_channels,out_channels,stride) -> None:
        super(SimpleResNetblock,self).__init__()

        self.conv1=nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=3,padding=1,stride=stride)
        self.bn1=nn.BatchNorm2d(out_channels)
        self.relu=nn.ReLU()

    def forward(self,x):
        x=self.conv1(x)
        x=self.bn1(x)
        x=self.relu(x)

        return x


In [78]:
class ResNet18(nn.Module):
    def __init__(self,block:SimpleResNetblock,in_channels,num_classes) -> None:
        super(ResNet18,self).__init__()

        self.conv1=nn.Conv2d(in_channels,64,kernel_size=7,stride=2,padding=3)
        self.bn1=nn.BatchNorm2d(64)
        self.relu=nn.ReLU()
        self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)

        self.conv2=block(in_channels=64,out_channels=128,stride=1)

        

        self.avg=nn.AvgPool2d((1,1))

        self.fc=nn.Linear(128*56*56,num_classes)


    def forward(self,x):
        x=self.conv1(x)
        x=self.bn1(x)
        x=self.relu(x)
        x=self.maxpool(x)

        x=self.conv2(x)

        x=self.avg(x)
        # print(f"shape after avg {x.shape}")
        x=x.reshape(x.shape[0],-1)
        # print(f"shape after reshape {x.shape}")
        x=self.fc(x)

        
        return x

In [80]:
net=ResNet18(SimpleResNetblock,3,3)
x=torch.randn(2,3,224,224)
print(f"in shape {x.shape}")

y=net(x)
print(f"out shape {y.shape}")

net_optimizer=torch.optim.SGD(net.parameters(),lr=0.1)
net_loss_fn=nn.CrossEntropyLoss()
res=Totrain(net,data,net_optimizer,net_loss_fn,3)






in shape torch.Size([2, 3, 224, 224])
out shape torch.Size([2, 3])


 10%|█         | 1/10 [00:04<00:38,  4.26s/it]

Epoch: 1 | train_loss: 167.4154 | test_loss: 501.8230 | 


 20%|██        | 2/10 [00:08<00:33,  4.22s/it]

Epoch: 2 | train_loss: 0.9553 | test_loss: 15.5252 | 


...

Epoch: 4 | train_loss: 1.0979 | test_loss: 1.5412 | 


 50%|█████     | 5/10 [00:20<00:20,  4.12s/it]

Epoch: 5 | train_loss: 1.5807 | test_loss: 2.7500 | 


 50%|█████     | 5/10 [00:23<00:23,  4.65s/it]


KeyboardInterrupt: 